In [54]:
import os
os.environ["KERAS_BACKEND"] = "torch"
import torch
import keras
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from keras.layers import TorchModuleWrapper
from keras.optimizers import Adam
import json
import pandas as pd
import pprint
from json import loads, dumps
import itertools
import random
from sklearn.model_selection import train_test_split
import pickle

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd drive/MyDrive/Colab Notebooks/NLP/project/Maven

/content/drive/MyDrive/Colab Notebooks/NLP/project/Maven


In [ ]:
!git clone https://github.com/THU-KEG/MAVEN-ERE.git

Cloning into 'MAVEN-ERE'...
remote: Enumerating objects: 177, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 177 (delta 10), reused 4 (delta 4), pack-reused 162 (from 1)
Receiving objects: 100% (177/177), 94.03 KiB | 972.00 KiB/s, done.
Resolving deltas: 100% (94/94), done.


In [ ]:
!wget -O MAVEN_ERE.zip https://cloud.tsinghua.edu.cn/f/a7d1db6c44ea458bb6f0/?dl=1
!unzip MAVEN_ERE.zip

--2024-11-23 21:09:03--  https://cloud.tsinghua.edu.cn/f/a7d1db6c44ea458bb6f0/?dl=1
Resolving cloud.tsinghua.edu.cn (cloud.tsinghua.edu.cn)... 101.6.15.69, 2402:f000:1:402:101:6:15:69
Connecting to cloud.tsinghua.edu.cn (cloud.tsinghua.edu.cn)|101.6.15.69|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cloud.tsinghua.edu.cn/seafhttp/files/f365834f-bfd4-46fc-9f2a-def2da4cba7b/MAVEN_ERE.zip [following]
--2024-11-23 21:09:04--  https://cloud.tsinghua.edu.cn/seafhttp/files/f365834f-bfd4-46fc-9f2a-def2da4cba7b/MAVEN_ERE.zip
Reusing existing connection to cloud.tsinghua.edu.cn:443.
HTTP request sent, awaiting response... 200 OK
Length: 19635238 (19M) [application/zip]
Saving to: ‘MAVEN_ERE.zip’

MAVEN_ERE.zip       100%[===================>]  18.73M  7.49MB/s    in 2.5s    

2024-11-23 21:09:07 (7.49 MB/s) - ‘MAVEN_ERE.zip’ saved [19635238/19635238]

Archive:  MAVEN_ERE.zip
   creating: MAVEN_ERE/
  inflating: MAVEN_ERE/test.jsonl    
  inflating: MAVEN

In [4]:
file_path = "/content/drive/MyDrive/Colab Notebooks/NLP/project/Maven/MAVEN_ERE/train.jsonl"
with open(file_path, 'r') as file:
    data = [json.loads(line) for line in file]

df = pd.DataFrame(data)



In [5]:
pprint.pp(data[0])

{'id': '364ed14fc610df6e25a2f446e2b2d2ab',
 'title': 'Expedition of the Thousand',
 'tokens': [['The',
             'Expedition',
             'of',
             'the',
             'Thousand',
             '(',
             'Italian',
             '``',
             'Spedizione',
             'dei',
             'Mille',
             "''",
             ')',
             'was',
             'an',
             'event',
             'of',
             'the',
             'Italian',
             'Risorgimento',
             'that',
             'took',
             'place',
             'in',
             '1860',
             '.'],
            ['a',
             'corps',
             'of',
             'volunteers',
             'led',
             'by',
             'giuseppe',
             'garibaldi',
             'sailed',
             'from',
             'quarto',
             ',',
             'near',
             'genoa',
             '(',
             'now',
             'quarto'

In [6]:
df.head(5)

,id,title,tokens,sentences,events,TIMEX,temporal_relations,causal_relations,subevent_relations
0,364ed14fc610df6e25a2f446e2b2d2ab,Expedition of the Thousand,"[[The, Expedition, of, the, Thousand, (, Itali...","[The Expedition of the Thousand (Italian ""Sped...",[{'id': 'EVENT_e2cbbfb4a5ab65856ccc7057443120e...,[{'id': 'TIME_c61b2c2b8b8c6656a1cc8443fed8c58a...,{'BEFORE': [['EVENT_8d2fbddb83f3c50eba8e24e215...,"{'CAUSE': [], 'PRECONDITION': [['EVENT_040b466...","[[EVENT_f535eb21f8f958a04cbc69c1d1521186, EVEN..."
1,0371bbf116422f8e3a0a853bdd1962aa,Murder of Leigh Leigh,"[[The, murder, of, Leigh, Leigh, ,, born, Leig...","[The murder of Leigh Leigh, born Leigh Rennea ...",[{'id': 'EVENT_36fe15f2ecceb14abdedc904976ed5a...,[{'id': 'TIME_626612ce2546f23317fe9c49b937db29...,{'BEFORE': [['EVENT_f69dc296dec6e31675b1a184ac...,"{'CAUSE': [], 'PRECONDITION': [['EVENT_d80b821...",[]
2,097b86ef2a4c5d21037a4c6b47b3a165,Hurricane Jerry (1989),"[[Hurricane, Jerry, caused, minor, damage, in,...",[Hurricane Jerry caused minor damage in Texas ...,[{'id': 'EVENT_70824ec59e32d4f9f1edc2ba7e80ab9...,[{'id': 'TIME_178172b7abeec20195e2faded4d1d1fa...,{'BEFORE': [['TIME_d33ce93f453ddf764f6c204e3d7...,{'CAUSE': [['EVENT_f444a52cf6695b576d0b7605584...,[]
3,ab70a3e49966caa8c35f8b27fabea3ad,Death of Christopher Alder,"[[Christopher, Alder, was, a, trainee, compute...",[Christopher Alder was a trainee computer prog...,[{'id': 'EVENT_3ccd4f275de7691f301ab49019a1f7c...,[{'id': 'TIME_c4630b85e20c70b6e70a457abab0d510...,{'BEFORE': [['EVENT_60986e24389de9cf1a8845358e...,{'CAUSE': [['EVENT_38fced5f4ff38829319f42024cb...,[]
4,49e16a75dfd748513d5db9f8a849d218,Clackamas Town Center shooting,"[[On, December, 11, ,, 2012, ,, a, shooting, o...","[On December 11, 2012, a shooting occurred at ...",[{'id': 'EVENT_422ed35102bdb680bc098aa3a94a724...,[{'id': 'TIME_26d49779da7be1d48e35f72cbfedb016...,{'BEFORE': [['EVENT_9590eefd428bcee365ddf8df71...,{'CAUSE': [['EVENT_2ba609b3aacac38e6c2b84e009a...,[]


In [7]:
len(data)

2913

In [8]:
events_data = []
for doc in data:
    doc_id = doc['id']
    for event in doc.get('events', []):
          events_data.append({
              'doc_id': doc_id,
              'event_id': event['id'],
              'event_type': event['type'],
              'event_type_id': event['type_id'],
              'mention_tw': event['mention'][0]['trigger_word'],
              'sent_id': event['mention'][0]['sent_id']
            })

events_df = pd.DataFrame(events_data)
events_df

,doc_id,event_id,event_type,event_type_id,mention_tw,sent_id
0,364ed14fc610df6e25a2f446e2b2d2ab,EVENT_e2cbbfb4a5ab65856ccc7057443120e0,Control,41,ruled,1
1,364ed14fc610df6e25a2f446e2b2d2ab,EVENT_0b0c5d1268cc135278489b7f0004c0c3,Achieve,127,achieve,6
2,364ed14fc610df6e25a2f446e2b2d2ab,EVENT_e71684bf9f6e7b50bf5316982a640c15,Creating,60,creation,3
3,364ed14fc610df6e25a2f446e2b2d2ab,EVENT_f535eb21f8f958a04cbc69c1d1521186,Self_motion,46,Expedition,0
4,364ed14fc610df6e25a2f446e2b2d2ab,EVENT_d2587256a67ce75ea9d9d864288d822c,Motion,10,landed,1
...,...,...,...,...,...,...
67979,eb724b2d3f906f8dd7aabc218adba5dd,EVENT_dffddcbaa25cc39d65c87b6f5caf2c06,Cause_change_of_position_on_a_scale,37,declined,11
67980,eb724b2d3f906f8dd7aabc218adba5dd,EVENT_7fab91a84c71e47b6d1a7dd0e2a9116e,Motion,10,drove,6
67981,eb724b2d3f906f8dd7aabc218adba5dd,EVENT_2e811b38d1aab386a2e752d2c2b983a6,Building,44,build,6
67982,eb724b2d3f906f8dd7aabc218adba5dd,EVENT_ddd69c03a2f138791516dc8926f412b1,Change_of_leadership,96,revolt,1


In [9]:
# Filter the DataFrame to see the events for the first doc
filtered_df = events_df[events_df["doc_id"] ==  "364ed14fc610df6e25a2f446e2b2d2ab"]
filtered_df

,doc_id,event_id,event_type,event_type_id,mention_tw,sent_id
0,364ed14fc610df6e25a2f446e2b2d2ab,EVENT_e2cbbfb4a5ab65856ccc7057443120e0,Control,41,ruled,1
1,364ed14fc610df6e25a2f446e2b2d2ab,EVENT_0b0c5d1268cc135278489b7f0004c0c3,Achieve,127,achieve,6
2,364ed14fc610df6e25a2f446e2b2d2ab,EVENT_e71684bf9f6e7b50bf5316982a640c15,Creating,60,creation,3
3,364ed14fc610df6e25a2f446e2b2d2ab,EVENT_f535eb21f8f958a04cbc69c1d1521186,Self_motion,46,Expedition,0
4,364ed14fc610df6e25a2f446e2b2d2ab,EVENT_d2587256a67ce75ea9d9d864288d822c,Motion,10,landed,1
5,364ed14fc610df6e25a2f446e2b2d2ab,EVENT_e12e909ff79793dc2b7b534d37685e9c,Process_start,31,instigated,5
6,364ed14fc610df6e25a2f446e2b2d2ab,EVENT_f652152ef7245a6f5c27396a2b1964e1,Process_end,61,concluded,3
7,364ed14fc610df6e25a2f446e2b2d2ab,EVENT_15451e1aabcb291d411af6a3d7e32911,Cause_to_amalgamate,131,unification,5
8,364ed14fc610df6e25a2f446e2b2d2ab,EVENT_6912f3e89bb068b0fdd78948b8cb9be2,Aiming,98,aiming,2
9,364ed14fc610df6e25a2f446e2b2d2ab,EVENT_93e0350c5f49e6ddb79b2091fda2012a,Bringing,78,brought,3


In [11]:
# order the filtered DataFrame by the values in sent_id (the index of the corresponding sentence in the doc)
new_sorted = filtered_df.sort_values(by='sent_id')
new_sorted

,doc_id,event_id,event_type,event_type_id,mention_tw,sent_id
3,364ed14fc610df6e25a2f446e2b2d2ab,EVENT_f535eb21f8f958a04cbc69c1d1521186,Self_motion,46,Expedition,0
15,364ed14fc610df6e25a2f446e2b2d2ab,EVENT_5d5130ba19c9706c4fff53d34225fb1c,Process_start,31,took place,0
0,364ed14fc610df6e25a2f446e2b2d2ab,EVENT_e2cbbfb4a5ab65856ccc7057443120e0,Control,41,ruled,1
18,364ed14fc610df6e25a2f446e2b2d2ab,EVENT_451b7cde13d2b8c21426db027c51096f,Self_motion,46,sailed,1
12,364ed14fc610df6e25a2f446e2b2d2ab,EVENT_c027e659d7fe424a0a57ecbe35b3a7f9,Conquering,21,conquer,1
4,364ed14fc610df6e25a2f446e2b2d2ab,EVENT_d2587256a67ce75ea9d9d864288d822c,Motion,10,landed,1
8,364ed14fc610df6e25a2f446e2b2d2ab,EVENT_6912f3e89bb068b0fdd78948b8cb9be2,Aiming,98,aiming,2
9,364ed14fc610df6e25a2f446e2b2d2ab,EVENT_93e0350c5f49e6ddb79b2091fda2012a,Bringing,78,brought,3
11,364ed14fc610df6e25a2f446e2b2d2ab,EVENT_e92e9b331fe3220ec0a33d8bd2489fd0,Ratification,144,plebiscite,3
2,364ed14fc610df6e25a2f446e2b2d2ab,EVENT_e71684bf9f6e7b50bf5316982a640c15,Creating,60,creation,3


# Use the sent_id to sort the list of triggered words (generated sequence)

In [12]:
# Filter and sort the whole DataFrame to obtain a dictionary where the keys are doc ids and the values are a list trigger_words found in the mentions for each event
events_dic ={}

index_list= []
event_types_labels= []

for index, row in events_df.iterrows():
  doc_id= events_df['doc_id'][index]
  if doc_id not in index_list:
    index_list.append(doc_id)
    filtered_df = events_df[events_df["doc_id"] ==  doc_id]
    # order the data frame by the values in sent_id
    sorted = filtered_df.sort_values(by='sent_id')
    mention_tws=sorted['mention_tw'].tolist() # converts column to list
    event_types_labels.append(sorted['event_type'].tolist())
    # add to dictionary
    events_dic[doc_id]=mention_tws


# The triggered words for each mention as the generated sequence.

In [13]:
# Get first N items in dictionary
pprint.pp( dict(itertools.islice(events_dic.items(), 3)) )

{'364ed14fc610df6e25a2f446e2b2d2ab': ['Expedition',
                                      'took place',
                                      'ruled',
                                      'sailed',
                                      'conquer',
                                      'landed',
                                      'aiming',
                                      'brought',
                                      'plebiscite',
                                      'creation',
                                      'concluded',
                                      'venture',
                                      'decided',
                                      'unification',
                                      'instigated',
                                      'bolster',
                                      'utilized',
                                      'participated',
                                      'achieve',
                                      'end',
    

# Use the event types as class labels

In [14]:
event_types_labels[:3]

[['Self_motion',
  'Process_start',
  'Control',
  'Self_motion',
  'Conquering',
  'Motion',
  'Aiming',
  'Bringing',
  'Ratification',
  'Creating',
  'Process_end',
  'Self_motion',
  'Deciding',
  'Cause_to_amalgamate',
  'Process_start',
  'Cause_to_make_progress',
  'Using',
  'Participation',
  'Achieve',
  'Process_end',
  'Dispersal'],
 ['Participation',
  'Committing_crime',
  'Coming_to_be',
  'Attack',
  'Name_conferral',
  'Self_motion',
  'Departing',
  'Know',
  'Damaging',
  'Bodily_harm',
  'Request',
  'Legal_rulings',
  'Assistance',
  'Releasing',
  'Request',
  'Request',
  'Judgment_communication',
  'Judgment_communication',
  'Competition',
  'Criminal_investigation',
  'Research',
  'Convincing',
  'Receiving',
  'Aiming',
  'Aiming',
  'Judgment_communication',
  'Adducing',
  'Influence',
  'Change',
  'Name_conferral',
  'Preventing_or_letting'],
 ['Causation',
  'Motion',
  'Damaging',
  'Coming_to_be',
  'Name_conferral',
  'Motion',
  'Cause_change_of_st

In [15]:
len(event_types_labels)

2913

# Find cause_effect pairs in the generated sequence from the causal relations.

In [16]:
# Filter and sort the whole DataFrame to obtain a dictionary where the keys are doc ids and the values are a list of tuples in the form (triggered_word, event_id)
events_dic_tuple={}

index_list= []
event_types_labels= []

for index, row in events_df.iterrows():
  doc_id= events_df['doc_id'][index]
  if doc_id not in index_list:
    index_list.append(doc_id)
    filtered_df = events_df[events_df["doc_id"] ==  doc_id]
    # order the data frame by the values in sent_id
    sorted = filtered_df.sort_values(by='sent_id')

    # create tuple (tw,event_id)
    mention_tws=sorted['mention_tw'].tolist() # converts column to list
    tws_event_ids= sorted['event_id'].tolist()
    tw_eID= zip(mention_tws,tws_event_ids)
    tuple_list= list(tw_eID)

    event_types_labels.append(sorted['event_type'].tolist())
    # add to dictionary
    events_dic_tuple[doc_id]=tuple_list

In [17]:
# Get first N items in dictionary
pprint.pp( dict(itertools.islice(events_dic_tuple.items(), 1)) )

{'364ed14fc610df6e25a2f446e2b2d2ab': [('Expedition',
                                       'EVENT_f535eb21f8f958a04cbc69c1d1521186'),
                                      ('took place',
                                       'EVENT_5d5130ba19c9706c4fff53d34225fb1c'),
                                      ('ruled',
                                       'EVENT_e2cbbfb4a5ab65856ccc7057443120e0'),
                                      ('sailed',
                                       'EVENT_451b7cde13d2b8c21426db027c51096f'),
                                      ('conquer',
                                       'EVENT_c027e659d7fe424a0a57ecbe35b3a7f9'),
                                      ('landed',
                                       'EVENT_d2587256a67ce75ea9d9d864288d822c'),
                                      ('aiming',
                                       'EVENT_6912f3e89bb068b0fdd78948b8cb9be2'),
                                      ('brought',
                         

In [18]:
causal_data = []
for doc in data:
    doc_id = doc['id']
    for relation_type, relations in doc.get('causal_relations', {}).items():
        for relation in relations:
            causal_data.append({
                'doc_id': doc_id,
                'relation_type': relation_type,
                'cause_event': relation[0],  # The first event ID in the relation
                'effect_event': relation[1]  # The second event ID in the relation
            })

causal_df = pd.DataFrame(causal_data)

In [19]:
causal_df

,doc_id,relation_type,cause_event,effect_event
0,364ed14fc610df6e25a2f446e2b2d2ab,PRECONDITION,EVENT_040b46602889e455df7b4327137034e9,EVENT_c027e659d7fe424a0a57ecbe35b3a7f9
1,364ed14fc610df6e25a2f446e2b2d2ab,PRECONDITION,EVENT_f535eb21f8f958a04cbc69c1d1521186,EVENT_5445faf0caf78e3dcc1dc88e3554fac8
2,364ed14fc610df6e25a2f446e2b2d2ab,PRECONDITION,EVENT_040b46602889e455df7b4327137034e9,EVENT_0b0c5d1268cc135278489b7f0004c0c3
3,364ed14fc610df6e25a2f446e2b2d2ab,PRECONDITION,EVENT_040b46602889e455df7b4327137034e9,EVENT_931d31a625729392d27925e697762d5f
4,364ed14fc610df6e25a2f446e2b2d2ab,PRECONDITION,EVENT_f535eb21f8f958a04cbc69c1d1521186,EVENT_0b0c5d1268cc135278489b7f0004c0c3
...,...,...,...,...
36311,eb724b2d3f906f8dd7aabc218adba5dd,PRECONDITION,EVENT_7f0177683d2e7ae323347c5f00534a36,EVENT_2e811b38d1aab386a2e752d2c2b983a6
36312,eb724b2d3f906f8dd7aabc218adba5dd,PRECONDITION,EVENT_7f0177683d2e7ae323347c5f00534a36,EVENT_7fab91a84c71e47b6d1a7dd0e2a9116e
36313,eb724b2d3f906f8dd7aabc218adba5dd,PRECONDITION,EVENT_7fab91a84c71e47b6d1a7dd0e2a9116e,EVENT_2e811b38d1aab386a2e752d2c2b983a6
36314,eb724b2d3f906f8dd7aabc218adba5dd,PRECONDITION,EVENT_a498cfc917fa127a2e996d0dcc61766a,EVENT_c901467686ab9b339f843fce73eed534


In [20]:
# Create a dictionary where the keys are the doc ids and the values are a list of causal relation tuples
cause_effect_dic ={}

index_list= []
cause_effect_tuples= []

for index, row in causal_df.iterrows():
  doc_id= causal_df['doc_id'][index]
  if doc_id not in index_list:
    index_list.append(doc_id)
    filtered_df = causal_df[causal_df["doc_id"] ==  doc_id]
    # create cause effect tuple
    cause_event=filtered_df['cause_event'].tolist() # converts column to list
    effect_event=filtered_df['effect_event'].tolist() # converts column to list
    c_e= list(zip(cause_event,effect_event))
    # add to dictionary
    cause_effect_dic[doc_id]=c_e

In [24]:
# Get first N items in dictionary
pprint.pp( dict(itertools.islice(cause_effect_dic.items(), 2)) )

{'364ed14fc610df6e25a2f446e2b2d2ab': [('EVENT_040b46602889e455df7b4327137034e9',
                                       'EVENT_c027e659d7fe424a0a57ecbe35b3a7f9'),
                                      ('EVENT_f535eb21f8f958a04cbc69c1d1521186',
                                       'EVENT_5445faf0caf78e3dcc1dc88e3554fac8'),
                                      ('EVENT_040b46602889e455df7b4327137034e9',
                                       'EVENT_0b0c5d1268cc135278489b7f0004c0c3'),
                                      ('EVENT_040b46602889e455df7b4327137034e9',
                                       'EVENT_931d31a625729392d27925e697762d5f'),
                                      ('EVENT_f535eb21f8f958a04cbc69c1d1521186',
                                       'EVENT_0b0c5d1268cc135278489b7f0004c0c3'),
                                      ('EVENT_040b46602889e455df7b4327137034e9',
                                       'EVENT_451b7cde13d2b8c21426db027c51096f'),
                      

In [22]:
# first doc in events
first_doc= events_dic_tuple["364ed14fc610df6e25a2f446e2b2d2ab"]
first_doc

[('Expedition', 'EVENT_f535eb21f8f958a04cbc69c1d1521186'),
 ('took place', 'EVENT_5d5130ba19c9706c4fff53d34225fb1c'),
 ('ruled', 'EVENT_e2cbbfb4a5ab65856ccc7057443120e0'),
 ('sailed', 'EVENT_451b7cde13d2b8c21426db027c51096f'),
 ('conquer', 'EVENT_c027e659d7fe424a0a57ecbe35b3a7f9'),
 ('landed', 'EVENT_d2587256a67ce75ea9d9d864288d822c'),
 ('aiming', 'EVENT_6912f3e89bb068b0fdd78948b8cb9be2'),
 ('brought', 'EVENT_93e0350c5f49e6ddb79b2091fda2012a'),
 ('plebiscite', 'EVENT_e92e9b331fe3220ec0a33d8bd2489fd0'),
 ('creation', 'EVENT_e71684bf9f6e7b50bf5316982a640c15'),
 ('concluded', 'EVENT_f652152ef7245a6f5c27396a2b1964e1'),
 ('venture', 'EVENT_931d31a625729392d27925e697762d5f'),
 ('decided', 'EVENT_8d2fbddb83f3c50eba8e24e215101547'),
 ('unification', 'EVENT_15451e1aabcb291d411af6a3d7e32911'),
 ('instigated', 'EVENT_e12e909ff79793dc2b7b534d37685e9c'),
 ('bolster', 'EVENT_040b46602889e455df7b4327137034e9'),
 ('utilized', 'EVENT_4381e133e1cf65fa2e5bc1767ed1d0db'),
 ('participated', 'EVENT_223bc4cd

In [25]:
len(events_dic_tuple["364ed14fc610df6e25a2f446e2b2d2ab"])

21

In [23]:
# firs doc in causal relations
first_causal= cause_effect_dic["364ed14fc610df6e25a2f446e2b2d2ab"]
first_causal

[('EVENT_040b46602889e455df7b4327137034e9',
  'EVENT_c027e659d7fe424a0a57ecbe35b3a7f9'),
 ('EVENT_f535eb21f8f958a04cbc69c1d1521186',
  'EVENT_5445faf0caf78e3dcc1dc88e3554fac8'),
 ('EVENT_040b46602889e455df7b4327137034e9',
  'EVENT_0b0c5d1268cc135278489b7f0004c0c3'),
 ('EVENT_040b46602889e455df7b4327137034e9',
  'EVENT_931d31a625729392d27925e697762d5f'),
 ('EVENT_f535eb21f8f958a04cbc69c1d1521186',
  'EVENT_0b0c5d1268cc135278489b7f0004c0c3'),
 ('EVENT_040b46602889e455df7b4327137034e9',
  'EVENT_451b7cde13d2b8c21426db027c51096f'),
 ('EVENT_8d2fbddb83f3c50eba8e24e215101547',
  'EVENT_931d31a625729392d27925e697762d5f'),
 ('EVENT_040b46602889e455df7b4327137034e9',
  'EVENT_562d52136def8fdfaa89dc3852695404'),
 ('EVENT_040b46602889e455df7b4327137034e9',
  'EVENT_f535eb21f8f958a04cbc69c1d1521186'),
 ('EVENT_040b46602889e455df7b4327137034e9',
  'EVENT_d2587256a67ce75ea9d9d864288d822c'),
 ('EVENT_f535eb21f8f958a04cbc69c1d1521186',
  'EVENT_562d52136def8fdfaa89dc3852695404'),
 ('EVENT_040b46602889

In [ ]:
len(cause_effect_dic["364ed14fc610df6e25a2f446e2b2d2ab"])

14

In [26]:
len(events_dic_tuple.keys())

2913

In [27]:
len(cause_effect_dic.keys())

2828

In [28]:
# function to find the tw causal_pairs in a list given one causal relation
def find_tuple(causal_tuple, twEID_tuple_list):
  ce_tuple=()
  val_tag=0
  cause=True
  effect=True
  for i in range (len(twEID_tuple_list)):
    ele= twEID_tuple_list[i]
    if (causal_tuple[0]==ele[1] and cause):
      cause_tw= ele[0]
      cause=False
      val_tag+=1
    if (causal_tuple[1]==ele[1] and effect):
      effect_tw= ele[0]
      effect=False
      val_tag+=1
    if val_tag==2:
      ce_tuple= (cause_tw,effect_tw)

  return ce_tuple


In [29]:
# first doc in events
first_doc= events_dic_tuple["364ed14fc610df6e25a2f446e2b2d2ab"]
first_doc

[('Expedition', 'EVENT_f535eb21f8f958a04cbc69c1d1521186'),
 ('took place', 'EVENT_5d5130ba19c9706c4fff53d34225fb1c'),
 ('ruled', 'EVENT_e2cbbfb4a5ab65856ccc7057443120e0'),
 ('sailed', 'EVENT_451b7cde13d2b8c21426db027c51096f'),
 ('conquer', 'EVENT_c027e659d7fe424a0a57ecbe35b3a7f9'),
 ('landed', 'EVENT_d2587256a67ce75ea9d9d864288d822c'),
 ('aiming', 'EVENT_6912f3e89bb068b0fdd78948b8cb9be2'),
 ('brought', 'EVENT_93e0350c5f49e6ddb79b2091fda2012a'),
 ('plebiscite', 'EVENT_e92e9b331fe3220ec0a33d8bd2489fd0'),
 ('creation', 'EVENT_e71684bf9f6e7b50bf5316982a640c15'),
 ('concluded', 'EVENT_f652152ef7245a6f5c27396a2b1964e1'),
 ('venture', 'EVENT_931d31a625729392d27925e697762d5f'),
 ('decided', 'EVENT_8d2fbddb83f3c50eba8e24e215101547'),
 ('unification', 'EVENT_15451e1aabcb291d411af6a3d7e32911'),
 ('instigated', 'EVENT_e12e909ff79793dc2b7b534d37685e9c'),
 ('bolster', 'EVENT_040b46602889e455df7b4327137034e9'),
 ('utilized', 'EVENT_4381e133e1cf65fa2e5bc1767ed1d0db'),
 ('participated', 'EVENT_223bc4cd

In [30]:
# firs doc in causal relations
first_causal= cause_effect_dic["364ed14fc610df6e25a2f446e2b2d2ab"]
first_causal[0]

('EVENT_040b46602889e455df7b4327137034e9',
 'EVENT_c027e659d7fe424a0a57ecbe35b3a7f9')

In [31]:
find_tuple(first_causal[0],first_doc)

('bolster', 'conquer')

In [32]:
events_keys= list(events_dic_tuple.keys())
causal_keys= list(cause_effect_dic.keys())
# Convert lists to sets
set1 = set(events_keys)
set2 = set(causal_keys)

# Find the difference using the symmetric_difference() method
difference = set1.symmetric_difference(set2)

# Convert the result back to a list if needed
difference_list = list(difference)

print(difference_list)
print(len(difference_list))

['96dc0e6b5c71fa510088bac73b9ae6a6', 'f2c4652deeeadbc9be5b57712d526693', '7b103d16894e4542a1c884e72be7f87d', 'c6c1fcdbe83432486767199ec462834f', '2290ebbc21222e2e2be77c3f4f092ad6', '4c68e98828acfba027c9cb570b565220', 'cf8ddc473e1df01e866e6310f75fa01b', '354064035625980895c554ab0808d695', '40da32bb369d3c04eee05cc40f133ca2', 'a94dd976894ac59064ed9749c41f6396', '05571bde1fbd05f6148916d68b0330b3', '620223923161a106bfe24f42df8fa80b', '851673b003a6270d39e2be324e5abd4e', '1fdb32f45e1e19bc74c64c27f60621c3', 'c233c8d5f8a7abe32fade6da89c97494', '3f16577b03532127e4a97bb3d83f6e4a', '28481e9ffea5feaefbdeecfa805bab7c', '4a6990a748f4ad7b130a951e8eedaf15', 'fc44c551f38dfed54ac75117e893908c', '5377a8a05f4025d8abc8164b378cbf15', '64f1ac7d4fdc69e8410b65a113a2e3d5', '2b15d0887b86c7fa0a3f5c18b44fae7f', '02cc26f3206a9f08ecc1914ce6319818', 'cdec7569e86f11e13ea443958455ef1d', '7dbe1e16c90c92fdf24daf9cad1a68f0', 'b5621244de0db5bf0f40f5f4df50a82f', '9c32bb8d72f91e8d85dc534ce19afc76', 'f92d72e268fc6948cd3be29e1a

In [33]:
# Create a dictionary where they keys are the document ids and the values are lists that contain tuples in the form (cause_tw,effect_tw)
tagged_tuple_dic= {}

tagged_list=[]
keys= list(events_dic_tuple.keys())
causal_values=list(cause_effect_dic.values())
index=0

for key,value in cause_effect_dic.items():
  if key in keys:
    tw_eventID_list= events_dic_tuple[keys[index]]
    for val in value:
      tagged= find_tuple(val,tw_eventID_list)
      tagged_list.append(tagged)
    ret = [tu for tu in tagged_list if tu]
    tagged_tuple_dic[keys[index]]= ret
    index+=1

# Causal_Pairs

In [34]:
# Get first N items in dictionary
pprint.pp( dict(itertools.islice(tagged_tuple_dic.items(), 3)) )

{'364ed14fc610df6e25a2f446e2b2d2ab': [('bolster', 'conquer'),
                                      ('Expedition', 'end'),
                                      ('bolster', 'achieve'),
                                      ('bolster', 'venture'),
                                      ('Expedition', 'achieve'),
                                      ('bolster', 'sailed'),
                                      ('decided', 'venture'),
                                      ('bolster', 'distribution'),
                                      ('bolster', 'Expedition'),
                                      ('bolster', 'landed'),
                                      ('Expedition', 'distribution'),
                                      ('bolster', 'end'),
                                      ('landed', 'conquer'),
                                      ('plebiscite', 'brought')],
 '0371bbf116422f8e3a0a853bdd1962aa': [('bolster', 'conquer'),
                                      ('Expedition', 'e

# Use the cause_effect pairs to generate the event streams.
  First, create train,test, dev splits

In [36]:
# Convert tuple to string
tu_str = ' '.join(tagged_tuple_dic['364ed14fc610df6e25a2f446e2b2d2ab'][0])
tu_str

'bolster conquer'

In [39]:
# Convert all tuples to string
for key,value in tagged_tuple_dic.items():
  new_value=[]
  for i in range(len(value)):
    tu_str = ' '.join(value[i])
    new_value.append(tu_str)
  tagged_tuple_dic[key]= new_value

In [40]:
# Get first N items in dictionary
pprint.pp( dict(itertools.islice(tagged_tuple_dic.items(), 3)) )

{'364ed14fc610df6e25a2f446e2b2d2ab': ['bolster conquer',
                                      'Expedition end',
                                      'bolster achieve',
                                      'bolster venture',
                                      'Expedition achieve',
                                      'bolster sailed',
                                      'decided venture',
                                      'bolster distribution',
                                      'bolster Expedition',
                                      'bolster landed',
                                      'Expedition distribution',
                                      'bolster end',
                                      'landed conquer',
                                      'plebiscite brought'],
 '0371bbf116422f8e3a0a853bdd1962aa': ['bolster conquer',
                                      'Expedition end',
                                      'bolster achieve',
                 

# Split Data into train,test,dev (60/20/20)

In [53]:
def split_dict(data, train_size=0.6, test_size=0.2, dev_size=0.2):
    """Splits a dictionary into train, test, and dev sets."""

    # Convert dictionary to list of key-value pairs
    items = list(data.items())

    # Shuffle the items randomly
    random.shuffle(items)

    # Split into train and test+dev
    train_items, test_dev_items = train_test_split(items, train_size=train_size)

    # Split test+dev into test and dev
    test_items, dev_items = train_test_split(test_dev_items, test_size=test_size / (test_size + dev_size))

    # Convert back to dictionaries
    train_data = dict(train_items)
    test_data = dict(test_items)
    dev_data = dict(dev_items)

    return train_data, test_data, dev_data

In [44]:
# Convert dictionary to DataFrame
MAVEN_ERE_train, MAVEN_ERE_test, MAVEN_ERE_dev= split_dict(tagged_tuple_dic,0.6,0.2,0.2)

In [46]:
# Get first N items in dictionary
pprint.pp( dict(itertools.islice(MAVEN_ERE_train.items(), 3)) )

{'83b5e7dd05eefe6221d84d5e624a71ba': ['bolster conquer',
                                      'Expedition end',
                                      'bolster achieve',
                                      'bolster venture',
                                      'Expedition achieve',
                                      'bolster sailed',
                                      'decided venture',
                                      'bolster distribution',
                                      'bolster Expedition',
                                      'bolster landed',
                                      'Expedition distribution',
                                      'bolster end',
                                      'landed conquer',
                                      'plebiscite brought',
                                      'occurred received',
                                      'entitled revised',
                                      'kicked left',
                  

In [48]:
len(tagged_tuple_dic['db9ec41549501a68d661c338443fca3d'])

591

In [49]:
len(MAVEN_ERE_train['db9ec41549501a68d661c338443fca3d'])

591

In [50]:
len(tagged_tuple_dic['364ed14fc610df6e25a2f446e2b2d2ab'])

14

In [51]:
len(MAVEN_ERE_train['364ed14fc610df6e25a2f446e2b2d2ab'])

14

# Save Pickle files

In [52]:
directory_path = '/content/drive/MyDrive/Colab Notebooks/NLP/project/PC-TES-PROJECT/causality-guided-Transformer/data/MAVEN_ERE/splits/'

In [56]:
# Save the MAVEN_ERE_train dictionary to a pickle file
with open(directory_path + 'maven_ere_seq_train.pkl', 'wb') as f:
    pickle.dump(MAVEN_ERE_train, f)

In [57]:
# Save the MAVEN_ERE_test dictionary to a pickle file
with open(directory_path + 'maven_ere_seq_test.pkl', 'wb') as f:
    pickle.dump(MAVEN_ERE_test, f)

In [58]:
# Save the MAVEN_ERE_dev dictionary to a pickle file
with open(directory_path + 'maven_ere_seq_dev.pkl', 'wb') as f:
    pickle.dump(MAVEN_ERE_dev, f)